In [1]:
import os
import requests
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import numpy as np
import pandas as pd
import seaborn as sns

# Test modules
from traceback import print_stack
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import *
import sys
sys.path.append(r'C:/Users/alexa/Desktop/PythonProjects/Cursos/Ironhack - Data Analytics/projects/final_project')
from utilities.custom_logger import CustomLogger
import logging

In [7]:
class Job():
    def __init__(self, position, company, location, posted_date, no_applicants, date_collected, type_workplace, 
                 required_skills, competitive_advantages, level, worktype, description):
        self.position = position
        self.company = company
        self.location = location
        self.posted_date = posted_date
        self.no_applicants = no_applicants
        self.date_collected = date_collected
        self.type_workplace = type_workplace
        self.required_skills = required_skills
        self.competitive_advantages = competitive_advantages
        self.level = level
        self.worktype = worktype
        self.description = description

class Company():
    def __init__(self, name, size, sector):
        self.name = name
        self.size = size
        self.sector = sector

class SeleniumDriver():
    log = CustomLogger(logging.DEBUG)

    def __init__(self, driver):
        options = webdriver.ChromeOptions()
        options.add_argument(r"--user-data-dir=C:\Users\alexa\AppData\Local\Google\Chrome\User Data\\")
        driver = webdriver.Chrome(executable_path='./resources/chromedriver.exe', options=options)
        return driver

    def get_by_type(self, locator_type):
        locator_type = locator_type.lower()
        if locator_type == 'id':
            return By.ID
        elif locator_type == 'name':
            return By.NAME
        elif locator_type == 'xpath':
            return By.XPATH
        elif locator_type == 'css':
            return By.CSS_SELECTOR
        elif locator_type == 'class':
            return By.CLASS_NAME
        elif locator_type == 'link':
            return By.LINK_TEXT
        else:
            self.log.info('Locator Type', locator_type, 'not correct/supported')
        return False
    
    def get_element(self, locator, locator_type='class'):
        element = None
        try:
            locator_type = locator_type.lower()
            by_type = self.get_by_type(locator_type)
            element = self.driver.find_element(by_type, locator)
            self.log.info('Element found with locator:', locator, '; and locator type:', locator_type)
        except:
            self.log.info('Element not found with locator:', locator, '; and locator type:', locator_type)
        return element
    
    def get_elements(self, locator, locator_type='class'):
        element = None
        try:
            locator_type = locator_type.lower()
            by_type = self.get_by_type(locator_type)
            element = self.driver.find_elements(by_type, locator)
            self.log.info('Element found with locator:', locator, '; and locator type:', locator_type)
        except:
            self.log.info('Element not found with locator:', locator, '; and locator type:', locator_type)
        return element
    
    def element_click(self, locator, locator_type='class'):
        try:
            element = self.get_element(locator, locator_type)
            element.click()
            self.log.info('Clicked on element with locator:', locator, '; and locator type:', locator_type)
        except:
            self.log.info('Could not click on element with locator:', locator, '; and locator type:', locator_type)
            print_stack()

    def send_keys(self, data, locator, locator_type='class'):
        try:
            element = self.get_element(locator, locator_type)
            element.send_keys(data)
            self.log.info('Sent data on element with locator:', locator, '; and locator type:', locator_type)
        except:
            self.log.info("Couldn't Sent data on element with locator:", locator, '; and locator type:', locator_type)
            print_stack()

    def is_element_present(self, locator, locator_type='class'):
        try:
            element = self.get_element(locator, locator_type)
            if element is not None:
                self.log.info('Element', locator, 'found')
                return True
            else:
                self.log.info('Element', locator, 'not found')
                return False
        except:
            self.log.info('Element', locator, 'not found')
            return False
        
    def wait_for_element(self, locator, locator_type='class', timeout=10, poll_frequency=0.5):
        element = None
        try:
            by_type = self.get_by_type(locator_type)
            self.log.info('Waiting for maximum ::', str(timeout), ':: seconds for element to be clickable')
            wait = WebDriverWait(self.driver, timeout, poll_frequency=poll_frequency, ignored_exceptions=[
                NoSuchElementException, ElementNotVisibleException, ElementNotSelectableException
            ])
            element = wait.until(EC.element_to_be_clickable((by_type, locator)))
            self.log.info('Element', locator, 'appeared on the web page.')
        except:
            self.log.info('Element not', locator, 'appeared on the web page.')
            print_stack()
        return element

class ChromeBrowser(SeleniumDriver):
    def __init__(self, name, size, sector):
        super.__init__(name, size, sector)


    def get_linkedin(driver, url, page=0):
        start = page * 25
        driver.get(f'{url}&start={start}')
        print(f'{url}+&start={start}')

In [ ]:
driver = SeleniumDriver()

# Data Extraction

Primeiramente, preciso extrair os dados de meu interesse para constituir um database bacana. Farei isso com um webscraper no Linkedin, de início, embora possa pensar em usar outras plataformas de emprego se necessário.

Entendendo o link:
    https://www.linkedin.com/jobs/search/?currentJobId=3571662289&keywords=analista%20de%20dados&refresh=true
    
Parâmetros GET:
- currentJobId
- keywords: o que está sendo pesquisado
- refresh

## Beautiful Soup

Para acessar a pesquisa de uma dada vaga, basta estar logado e entrar no link gerado

In [4]:
MAIN_LINKEDIN_LINK = 'https://www.linkedin.com/jobs/search/?'
searched_job = 'analista de dados'
keywords = 'keywords=' + searched_job
location = 'location=Brasil'
final_link = MAIN_LINKEDIN_LINK + keywords + '&' + location + '&' + 'geoId=106057199'

Agora, precisamos acessar com o BeautifulSoup e começar a coletar os dados!

In [5]:
response = requests.get(final_link)
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
soup.div.get_attribute_list

<bound method Tag.get_attribute_list of <div data-tracking-control-name="public_jobs_google-one-tap" id="google-one-tap__container"></div>>

In [7]:
lateral = soup.find('main')
results = lateral.find_all('li')
for result in results:
    print(result.span.get_text().strip())

Analista de Dados Jr
Vem ser ituber! Carreira de Dados 🧡
Analista de Dados | BI - Jr.
Analista de Dados - Soluções de Investimentos
Analista de Analytics JR
Analista de Dados e Analytics JR- Comercial- Vaga Exclusiva Para Profissionais com Deficiência
Analista de Dados Júnior
Analista de Dados Junior
Analista de Dados Júnior.
Analista de Dados JR
Analista de Dados/BI
Data Analyst Jr/ Analista de Dados Jr
Analista de BI I
Jr Data Analyst
Analista de Dados Jr.
Analista em gestão do conhecimento  (monitoramento de pesquisas e dados)
Analista de BI - Jr
Analista de dados (BI) Jr
Analista de Dados Júnior (Híbrido)
Analista de Dados de Prevenção a Fraude -  Sênior
ANALISTA DE DADOS JÚNIOR
Analista de Dados Junior
Estágio em BI
Analista Inteligência de Dados Pleno
Analista de dados (BI) Jr - Ribeirão Preto


In [8]:
final_link

'https://www.linkedin.com/jobs/search/?keywords=analista de dados&location=Brasil&geoId=106057199'

Infelizmente, o BeautifulSoup não consegue nos retornar a página correta, exigindo o uso do Selenium

## Selenium

Primeiro, instanciamos o WebDriver do Selenium. O parâmetro 'options' terá o argumento para guardar os dados da sessão, facilitando futuras utilizações e evitando a necessidade de repetidos logins. Em seguida, acessamos o link do linkedin.

In [9]:


def get_element(element, attribute_value, by_type):
    if by_type == 'class':
        if len(element.find_elements(By.CLASS_NAME, attribute_value)) > 0:
            return element.find_element(By.CLASS_NAME, attribute_value).get_attribute('innerText')
    elif by_type == 'tag':
        if len(element.find_elements(By.TAG_NAME, attribute_value)) > 0:
            return element.find_element(By.TAG_NAME, attribute_value).get_attribute('innerText')
    elif by_type == 'id':
        if len(element.find_elements(By.ID, attribute_value)) > 0:
            return element.find_element(By.ID, attribute_value).get_attribute('innerText')
    return np.nan

def calculate_date(quantity, type_of_date):
    if 'minuto' in type_of_date:
        return datetime.today() - timedelta(minutes=quantity)
    elif 'hora' in type_of_date:
        return datetime.today() - timedelta(hours=quantity)
    elif 'dia' in type_of_date:
        return datetime.today() - timedelta(days=quantity)
    elif 'semana' in type_of_date:
        return datetime.today() - timedelta(weeks=quantity)
    elif 'mes' in re.sub('ê', 'e', type_of_date):
        return datetime.today() - relativedelta(months=quantity)
    return np.nan

def get_data_from_linkedin_page(driver, limit=False, limit_qtd=5):
    # Get job data, scroll down to load every job, get the data again
    job_list = driver.find_element(By.CLASS_NAME, 'scaffold-layout__list-container')
    job_list = job_list.find_elements(By.TAG_NAME, 'a')
    for i in range(10):
        job_list[0].send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    job_list = driver.find_element(By.CLASS_NAME, 'scaffold-layout__list-container')
    job_list = job_list.find_elements(By.TAG_NAME, 'a')

    # Getting the data for the dictionary
    job_collection = []
    i=0

    for job in job_list:
    
        job_data = dict()
        job.click()
        time.sleep(1.5)
        job_content = driver.find_element(By.CLASS_NAME, 'jobs-unified-top-card__content--two-pane')

        # First area of information (top)
        job_data['title'] = get_element(job_content, 'h2', 'tag')
        job_data['company'] = get_element(job_content, 'jobs-unified-top-card__company-name', 'class')
        job_data['location'] = get_element(job_content, 'jobs-unified-top-card__bullet', 'class')
        job_data['type_workplace'] = get_element(job_content, 'jobs-unified-top-card__workplace-type', 'class')
        job_data['applicant_count'] = get_element(job_content, 'jobs-unified-top-card__applicant-count', 'class')
        if isinstance(job_data['applicant_count'], str):
            job_data['applicant_count'] = job_data['applicant_count'].split()[0]

        # Calculating posted date
        if len(job_content.find_elements(By.CLASS_NAME, 'jobs-unified-top-card__posted-date')) > 0:
            quantity, temporal_type = get_element(job_content, 'jobs-unified-top-card__posted-date', 'class').split()[1:]
            job_data['posted_date'] = calculate_date(int(quantity), temporal_type)
        
        # Second area of information (job insight)
        try:
            job_insight = job_content.find_element(By.CLASS_NAME, 'mt5')
            job_insights = job_insight.find_elements(By.TAG_NAME, 'li')
            job_insights = [insight.get_attribute('innerText') for insight in job_insights]
            for insight in job_insights:
                if 'competências' in insight.lower():
                    job_data['skills'] = insight.split(': ')[1]
            job_data['worktype'] = job_insights[0].split('·')[0].strip()
            if len(job_insights[0].split('·')) > 1:
                job_data['level'] = job_insights[0].split('·')[1].strip()
            job_data['company_size'] = job_insights[1]
            
        except Exception as e:
            print('erro:', job_data['title'], '-', e)

        # Main job content:
        job_data['about_job'] = get_element(driver, 'job-details', 'id')


        job_collection.append(job_data)
        if limit:
            i+=1
            if i == limit_qtd:
                break
    
    return job_collection

A página de vagas do Linkedin é dividida em dois painéis, um com a lista de vagas e outra com a descrição da vaga selecionada, começando a partir da primeira. A lista de vagas é carregada na medida em que descemos por ela, então o comando .execute_script irá fazer um scroll down para carregarmos todas as vagas da primeira página.

Em seguida, guardamos todas as vagas numa lista de WebElements.

In [10]:
driver = open_navigator(final_link)
all_jobs = []
for page in range(40):
    get_linkedin(driver, final_link, page)
    all_jobs.extend(get_data_from_linkedin_page(driver))

C:\Users\alexa\AppData\Local\Temp\ipykernel_17680\315899760.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./resources/chromedriver.exe', options=options)


https://www.linkedin.com/jobs/search/?keywords=analista de dados&location=Brasil&geoId=106057199+&start=0
https://www.linkedin.com/jobs/search/?keywords=analista de dados&location=Brasil&geoId=106057199+&start=25
https://www.linkedin.com/jobs/search/?keywords=analista de dados&location=Brasil&geoId=106057199+&start=50
https://www.linkedin.com/jobs/search/?keywords=analista de dados&location=Brasil&geoId=106057199+&start=75
https://www.linkedin.com/jobs/search/?keywords=analista de dados&location=Brasil&geoId=106057199+&start=100
erro: Programador - list index out of range
https://www.linkedin.com/jobs/search/?keywords=analista de dados&location=Brasil&geoId=106057199+&start=125
https://www.linkedin.com/jobs/search/?keywords=analista de dados&location=Brasil&geoId=106057199+&start=150
https://www.linkedin.com/jobs/search/?keywords=analista de dados&location=Brasil&geoId=106057199+&start=175
erro: Desenvolvedor jr. - list index out of range
https://www.linkedin.com/jobs/search/?keywords=

In [12]:
df_jobs = pd.DataFrame(all_jobs)

In [13]:
df_jobs.head()

,title,company,location,type_workplace,applicant_count,posted_date,skills,worktype,level,company_size,about_job
0,"Data Analyst (Bangkok Based, Relocation Provided)",Agoda,"Porto Alegre, Rio Grande do Sul, Brasil",NaN,4,2023-05-26 16:01:22.414820,"Comunicação, Capacidade de organização, e mais 8",Tempo integral,Júnior,"5.001-10.000 funcionários · Tecnologia, Inform...",Sobre a vaga\nAbout Agoda\n\n\n\n\nAgoda is an...
1,Football Statistician,Genius Sports,"Iporá, Goiás, Brasil",Presencial,43,2023-05-23 16:01:24.447378,"Esportes, Inglês, e mais 8",Contrato,Assistente,1.001-5.000 funcionários · Desenvolvimento de ...,Sobre a vaga\n\nLove sports?\n\n\n\n\nWe're lo...
2,Football Statistician,Genius Sports,"Tocantinópolis, Tocantins, Brasil",Presencial,25,2023-05-29 16:01:26.395033,"Coleta de dados, Futebol americano, e mais 8",Contrato,Assistente,1.001-5.000 funcionários · Desenvolvimento de ...,Sobre a vaga\n\nLove sports?\n\n\n\n\nWe're lo...
3,"Statistical Analyst (Bangkok Based, Relocation...",Agoda,"Brasília, Distrito Federal, Brasil",NaN,10,2023-05-26 16:01:31.199296,"Comunicação, Capacidade de organização, e mais 8",Tempo integral,Júnior,"5.001-10.000 funcionários · Tecnologia, Inform...",Sobre a vaga\nAbout Agoda\n\n\n\n\nAgoda is an...
4,Analista de BI,Bacio di Latte,"São Paulo, São Paulo, Brasil",Presencial,NaN,2023-05-29 16:01:36.172219,"Comunicação, Relatórios e análises, e mais 8",Contrato,Pleno-sênior,1.001-5.000 funcionários · Serviços de aliment...,Sobre a vaga\n\nBuscamos sempre o que há de me...


In [14]:
df_jobs.shape

(975, 11)

In [264]:
df_jobs.location.unique()
df_jobs.location = df_jobs.location.str.strip()

In [24]:
df_jobs2 = df_jobs.copy()

In [25]:
df_jobs2['about_job'] = df_jobs['about_job'].str.replace('\n', ' - ')

In [26]:
df_jobs2.to_csv('df_jobs_v1.csv', index=None, sep=',')

In [27]:
a = pd.read_csv('df_jobs_v1.csv', sep=',')

In [30]:
a.shape

(975, 11)

In [31]:
a.head()

,title,company,location,type_workplace,applicant_count,posted_date,skills,worktype,level,company_size,about_job
0,"Data Analyst (Bangkok Based, Relocation Provided)",Agoda,"Porto Alegre, Rio Grande do Sul, Brasil",NaN,4.0,2023-05-26 16:01:22.414820,"Comunicação, Capacidade de organização, e mais 8",Tempo integral,Júnior,"5.001-10.000 funcionários · Tecnologia, Inform...",Sobre a vaga - About Agoda - - - - - Agoda...
1,Football Statistician,Genius Sports,"Iporá, Goiás, Brasil",Presencial,43.0,2023-05-23 16:01:24.447378,"Esportes, Inglês, e mais 8",Contrato,Assistente,1.001-5.000 funcionários · Desenvolvimento de ...,Sobre a vaga - - Love sports? - - - - - W...
2,Football Statistician,Genius Sports,"Tocantinópolis, Tocantins, Brasil",Presencial,25.0,2023-05-29 16:01:26.395033,"Coleta de dados, Futebol americano, e mais 8",Contrato,Assistente,1.001-5.000 funcionários · Desenvolvimento de ...,Sobre a vaga - - Love sports? - - - - - W...
3,"Statistical Analyst (Bangkok Based, Relocation...",Agoda,"Brasília, Distrito Federal, Brasil",NaN,10.0,2023-05-26 16:01:31.199296,"Comunicação, Capacidade de organização, e mais 8",Tempo integral,Júnior,"5.001-10.000 funcionários · Tecnologia, Inform...",Sobre a vaga - About Agoda - - - - - Agoda...
4,Analista de BI,Bacio di Latte,"São Paulo, São Paulo, Brasil",Presencial,NaN,2023-05-29 16:01:36.172219,"Comunicação, Relatórios e análises, e mais 8",Contrato,Pleno-sênior,1.001-5.000 funcionários · Serviços de aliment...,Sobre a vaga - - Buscamos sempre o que há de ...
